In [1]:
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("PINECONE_API_KEY")

pc = Pinecone(api_key=api_key)
index = pc.Index("empower-me")

/Users/akashwudali/Documents/MedHacks/venv/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [11]:
import itertools
import time
import tqdm
from sentence_transformers import SentenceTransformer
from pprint import pprint

import json
import os



# this method is more memory efficient that regular for loop since it only acces memory in chunks as needed

def generateEmbeddings(embeddingString):
    model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
    embeddings = model.encode([embeddingString])
    return embeddings[0]

def databaseBatchInsert(dct, batchSize, namespace):
    it = iter(dct.keys())
    chunk = itertools.islice(it, batchSize)
    while chunk:
        vectors = []
        for key in chunk:
            embedding = generateEmbeddings(dct[key]["embeddingString"])
            vectors.append({"id":key, "values":embedding, "metadata":dct[key]})
        index.upsert(vectors, namespace=namespace)
        chunk = itertools.islice(it, batchSize)
        print("Successfully Added Vector")

filepath = "parsedEhrData/"
data = []
for pfile in os.listdir(filepath):   
    with open(filepath + pfile, 'r') as file:
        namespace = pfile.replace(".json", "")
        data.append((namespace, json.load(file)))

for namespace, dct in data:
    print(namespace)
    databaseBatchInsert(dct, 200, namespace)

practitionerInfo


KeyboardInterrupt: 